In [32]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re

page = 1
BASE_URL = 'https://bhaskarlive.in/'
country = 'bangladesh'
initial_url = f'{BASE_URL}page/{page}/?s={country}'



#USE WHITE LOOP TO FIND ALL PAGES.
#AT THE TIME OF SCRPAING, THERE ARE 200 PAGES.

#IN THIS WEBSITE, ALL PAGES NEED TO BE LOADED USING SELENIUM FIRST AND ONLY THEN IT'S POSSIBLE TO SCRAPE USING BEAUTIFULSOUP.

#TAKING FIRST 3 PAGES FOR SCRPAING.



all_news_links= []

for page in range(1,4):
    
    initial_url = f'{BASE_URL}page/{page}/?s={country}'

    driver = webdriver.Chrome()
    driver.get(initial_url)
    
    time.sleep(10)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    main_div = soup.find('div', class_ = 'news-list')

    if main_div:

        all_articles = main_div.find_all('article')

        for each_article in all_articles:

            all_news_links.append(each_article.find('a').get('href'))
    else:
        print('no main div')
    
    
    driver.quit()
    

#SCRAPING ALL NEWS LINKS

counter = 0
data_list = []

for url in all_news_links[:10]:
    
    
        driver = webdriver.Chrome()
        driver.get(url)

        time.sleep(10)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        #TITLE

        title_tag = soup.find('h1', class_ = 'post-tile entry-title')
        title = title_tag.text if title_tag else 'Title not found'
        title_translation = 'None'

        #DATE

        date_info = soup.find('time', class_ = 'updated')

        if date_info:

            date_data = date_info.get('datetime')

            only_date = date_data.split('T')[0]
            only_time = date_data.split('T')[1]
            time_ = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
            cleaned_date = f"{only_date},{time_}"

            source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M") + timedelta(hours = 5, minutes = 30) 
            bangladesh_localtime = source_localtime + timedelta(minutes=30)

        else:

            date_data = 'Date data not found'


        #NO CONTENT SUMMARIES IN THIS WEBSITE
        content_summary = 'None'
        summary_translation = 'None'

        #CONTENT

        content = []

        main_content = soup.find('div', class_ = 'entry-content')

        if main_content:

            all_paras = main_content.find_all('p')

            for each_para in all_paras:
                content.append(each_para.text)

            content.pop()
            content.pop()

            author = content[-1]
            author = re.sub(r'[^\w]', ' ', author)
            author = author.strip()

            content.pop()

            full_content = ''.join(content)



        else:

            full_content = 'Content not found'
            author = 'Author not found'

        content_translation = 'None'
        
        
        
        
        data_dict = {
            "url": url,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }

        counter+=1


        if (full_content != "Content Not Found" and title != 'Title not found'):

                if data_dict not in data_list:
                        # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')
                
        
        driver.quit()


df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_Bhaskar_Live.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="bangladesh_localtime", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://bhaskarlive.in/charith-asalanka-to-lea...,Charith Asalanka to lead Sri Lanka in first tw...,"Colombo, Feb 28 (IANS) Sri Lanka Cricket (SLC)...",None,None,None,None,IANS,bangladesh,2024-02-28 19:33:00,2024-02-28 20:03:00
1,https://bhaskarlive.in/vlaeminck-molineux-retu...,"Vlaeminck, Molineux returns as Australia name ...","Melbourne, Feb 27 (IANS) Australia have recall...",None,None,None,None,IANS,bangladesh,2024-02-27 10:27:00,2024-02-27 10:57:00
2,https://bhaskarlive.in/bangladesh-embraces-ind...,Bangladesh embraces Indian medico’s ‘Save Girl...,"Pune, Feb 25 (IANS) Thousands of people and me...",None,None,None,None,IANS,bangladesh,2024-02-25 11:12:00,2024-02-25 11:42:00
3,https://bhaskarlive.in/bangladesh-plans-to-pro...,Bangladesh plans to produce FMD vaccine for li...,"Dhaka, Feb 23 (IANS) Bangladeshi Fisheries and...",None,None,None,None,IANS,bangladesh,2024-02-23 15:27:00,2024-02-23 15:57:00
4,https://bhaskarlive.in/bengal-ration-case-ed-p...,Bengal ration case: ED probes Bangladesh links...,"Kolkata, Feb 16 (IANS) Enforcement Directorate...",None,None,None,None,IANS,bangladesh,2024-02-16 15:13:00,2024-02-16 15:43:00
